In [1]:
import torch as th
import numpy as np
import pickle
import torchvision
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import numpy as np
from pytorch_lightning import seed_everything
import torch as th
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import sys
import glob
sys.path.insert(0, '../../sample_scripts/')

from sample_scripts.sample_utils.inference_utils import to_tensor
from sample_scripts.sample_utils.vis_utils import plot_image
from sample_scripts.sample_utils import (
    ckpt_utils, 
    file_utils,
    params_utils,
)
from guided_diffusion.dataloader.img_deca_datasets import load_data_img_deca

device = 'cuda:1'

plt.rcParams["savefig.bbox"] = 'tight'
def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
        
def get_cfg(self):
    from config.base_config import parse_args
    cfg_file_path = glob.glob("/home/mint/guided-diffusion/config/*/*", recursive=True)
    cfg_file_path = [cfg_path for cfg_path in cfg_file_path if f"/{self.cfg_name}" in cfg_path]    # Add /{}/ to achieve a case-sensitive of folder
    print("[#] Config Path : ", cfg_file_path)
    assert len(cfg_file_path) <= 1
    assert len(cfg_file_path) > 0
    cfg_file = cfg_file_path[0]
    cfg = parse_args(ipynb={'mode':True, 'cfg':cfg_file})
    return cfg

In [2]:
ckpt_loader = ckpt_utils.CkptLoader(log_dir="UNetCond_Spatial_Hadamart_Tanh_Shape", cfg_name="UNetCond_Spatial_Hadamart_Tanh_Shape.yaml")
cfg = ckpt_loader.cfg
cfg.img_cond_model.in_image = cfg.img_cond_model.in_image + ['faceseg_bg_noface&nohair']
cfg.img_cond_model.prep_image = [None, None]
# Load dataset
img_dataset_path = f"/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/"
deca_dataset_path = f"/data/mint/DPM_Dataset/ffhq_256_with_anno/params/"

loader, dataset, _ = load_data_img_deca(
    data_dir=img_dataset_path,
    deca_dir=deca_dataset_path,
    batch_size=int(1e7),
    image_size=cfg.img_model.image_size,
    deterministic=cfg.train.deterministic,
    augment_mode=cfg.img_model.augment_mode,
    resize_mode=cfg.img_model.resize_mode,
    in_image_UNet=cfg.img_model.in_image,
    params_selector=cfg.param_model.params_selector + ['albedo'],
    rmv_params=cfg.param_model.rmv_params,
    set_='valid',
    cfg=cfg,
)

# _, _, avg_dict = load_data_img_deca(
#     data_dir=img_dataset_path,
#     deca_dir=deca_dataset_path,
#     batch_size=int(1e7),
#     image_size=cfg.img_model.image_size,
#     deterministic=cfg.train.deterministic,
#     augment_mode=cfg.img_model.augment_mode,
#     resize_mode=cfg.img_model.resize_mode,
#     in_image_UNet=cfg.img_model.in_image,
#     params_selector=cfg.param_model.params_selector + ['albedo'],
#     rmv_params=cfg.param_model.rmv_params,
#     set_='train',
#     cfg=cfg,
# )

img_path = file_utils._list_image_files_recursively(f"{img_dataset_path}/valid/")
img_idx = file_utils.search_index_from_listpath(list_path=img_path, search=['60065.jpg', '60182.jpg'])
dat = th.utils.data.Subset(dataset, indices=img_idx)
subset_loader = th.utils.data.DataLoader(dat, batch_size=2,
                                    shuffle=False, num_workers=24)
                            
dat, model_kwargs = next(iter(subset_loader))

[#] Model Path :  ['/data/mint/model_logs_mount/v12/UNetCond_Spatial_Hadamart_Tanh_Shape/']
[#] Config Path :  ['/home/mint/guided-diffusion/config/EncLight_Spatial_Rendered_Face_Hadamart_clip/UNetCond_Spatial_Hadamart_Tanh_Shape.yaml']
Merging with :  Namespace(cfg='/home/mint/guided-diffusion/config/EncLight_Spatial_Rendered_Face_Hadamart_clip/UNetCond_Spatial_Hadamart_Tanh_Shape.yaml')



Loading deca params...: 100%|██████████| 9/9 [00:06<00:00,  1.30it/s]


[#] Parameters Conditioning
Params keys order :  ['shape', 'pose', 'exp', 'cam', 'faceemb']
Remove keys :  []


In [3]:
from importlib import reload
reload(params_utils)

rendered_image, orig_visdict = params_utils.render_deca(deca_params=model_kwargs, idx=0, n=2, useTex=True, extractTex=True, deca_mode='', use_detail=True)
grid = torchvision.utils.make_grid(orig_visdict['rendered_images'].mul(255).add_(0.5).clamp_(0, 255).cpu())
show(grid/255.0)

grid = torchvision.utils.make_grid(orig_visdict['rendered_images_uv_texture'].mul(255).add_(0.5).clamp_(0, 255).cpu())
show(grid/255.0)

grid = torchvision.utils.make_grid(orig_visdict['shape_images'].mul(255).add_(0.5).clamp_(0, 255).cpu())
show(grid/255.0)

grid = torchvision.utils.make_grid(model_kwargs['raw_image'].cpu())
show(grid/255.0)


creating the FLAME Decoder
trained model found. load /home/mint/guided-diffusion/sample_scripts/cond_utils/DECA/data/deca_model.tar


RuntimeError: Ninja is required to load C++ extensions

In [4]:
print(orig_visdict['rendered_images'].shape)
print(orig_visdict['shape_images'].shape)
print(orig_visdict['rendered_images_uv_texture'].shape)

torch.Size([2, 3, 256, 256])
torch.Size([2, 3, 256, 256])
torch.Size([2, 3, 224, 224])


In [2]:
from model_3d.FLAME.config import cfg as flame_cfg
import model_3d.FLAME.FLAME as FLAME
import model_3d.FLAME.utils.renderer as renderer_lib
import model_3d.FLAME.utils.util as util
from importlib import reload
reload(renderer_lib)
reload(FLAME)

flame = FLAME.FLAME(flame_cfg.model).cuda()
renderer = renderer_lib.SRenderY(image_size=256, obj_filename=flame_cfg.model.topology_path, uv_size=flame_cfg.model.uv_size).cuda()
print(renderer.faces.shape)
print(renderer.face_colors.shape)

def index_masking_faces(mask, faces):
    print("[#] Create indexing masked of faces...")
    import tqdm, pickle
    rm_faces = {}
    for k in mask.keys():
        rm_faces[k] = []
        print("Processing : ", k)
        for m in tqdm.tqdm(mask[k].tolist()):
            for i, f in enumerate(faces[0]):
                if m in f:
                    rm_faces[k].append(i)
        rm_faces[k] = np.array(rm_faces[k])
        break
    
    with open('faces_index.pkl', 'wb') as output:
        pickle.dump(rm_faces, output)

mask = np.load('./FLAME_masks.pkl', allow_pickle=True, encoding='latin1')
count = 0
mask_list = []
for k, v in mask.items():
    count += len(v)
    print(f"{k} - {v.shape}")
    mask_list.append(v)
mask_list = np.concatenate(mask_list, -1)

print(renderer.faces.shape)
print(type(mask['face'][0]))
print("#"*50)
index_masking_faces(mask, renderer.faces)

creating the FLAME Decoder


/home/mint/miniconda3/envs/3dr_sampling_deca/lib/python3.8/site-packages/pytorch3d/io/obj_io.py:457: UserWarning: Mtl file does not exist: /home/mint/guided-diffusion/model_3d/FLAME/data/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


torch.Size([1, 9976, 3])
torch.Size([1, 9976, 3, 3])


NameError: name 'np' is not defined

In [3]:
from importlib import reload
def params_to_model(shape, exp, pose, cam, lights, mask):
    
    from model_3d.FLAME.config import cfg as flame_cfg
    import model_3d.FLAME.FLAME as FLAME
    import model_3d.FLAME.utils.renderer as renderer_lib
    import model_3d.FLAME.utils.util as util
    reload(renderer_lib)
    reload(FLAME)

    flame = FLAME.FLAME(flame_cfg.model).cuda()
    verts, landmarks2d, landmarks3d = flame(shape_params=shape, 
            expression_params=exp, 
            pose_params=pose)
    renderer = renderer_lib.SRenderY(image_size=256, obj_filename=flame_cfg.model.topology_path, uv_size=flame_cfg.model.uv_size).cuda()

    verts = verts[:, mask['v_mask'], :]
    ## projection
    landmarks2d = util.batch_orth_proj(landmarks2d, cam)[:,:,:2]; landmarks2d[:,:,1:] = -landmarks2d[:,:,1:]#; landmarks2d = landmarks2d*self.image_size/2 + self.image_size/2
    landmarks3d = util.batch_orth_proj(landmarks3d, cam); landmarks3d[:,:,1:] = -landmarks3d[:,:,1:] #; landmarks3d = landmarks3d*self.image_size/2 + self.image_size/2
    trans_verts = util.batch_orth_proj(verts, cam); trans_verts[:,:,1:] = -trans_verts[:,:,1:]

    ## rendering
    # shape_images = renderer.render_shape(verts, trans_verts, lights=lights)
    v, f = renderer.render_shape(verts, trans_verts, lights=lights, mask=mask)
    return v, f
    print(v.shape, f.shape)
    assert False

    # opdict = {'verts' : verts,}
    # os.makedirs('./rendered_obj', exist_ok=True)
    # save_obj(renderer=renderer, filename=(f'./rendered_obj/{i}.obj'), opdict=opdict)
    shape_images = shape_images.mul(255).add_(0.5).clamp_(0, 255)
    return {"shape_images":shape_images, "landmarks2d":landmarks2d, "landmarks3d":landmarks3d}

v_mask = [10, 11, 12, 13, 14, 15, 111, 112, 219, 220, 221, 222, 372, 373, 374, 375, 462, 463, 496, 497, 552, 553, 558, 559, 563, 564, 649, 650, 736, 737, 784, 795, 1210, 1211, 1212, 1213, 1325, 1326, 1359, 1360, 1386, 1726, 1727, 1759, 1790, 1886, 1898, 1901, 1931, 1932, 1933, 1934, 1940, 1941, 1948, 1949, 2036, 2115, 2149, 2150, 2151, 2162, 2218, 2219, 2251, 2254, 2483, 2484, 2531, 2870, 2893, 2964, 2976, 2979, 3012, 3013, 3142, 3174, 3184, 3185, 3186, 3187, 3188, 3189, 3190, 3191, 3192, 3193, 3194, 3195, 3196, 3197, 3198, 3199, 3200, 3201, 3202, 3203, 3204, 3205, 3206, 3207, 3208, 3209, 3210, 3211, 3212, 3213, 3214, 3215, 3220, 3222, 3223, 3231, 3232, 3233, 3234, 3235, 3236, 3237, 3238, 3239, 3240, 3241, 3242, 3243, 3244, 3245, 3246, 3247, 3250, 3251, 3253, 3254, 3263, 3264, 3265, 3266, 3267, 3268, 3269, 3270, 3275, 3276, 3277, 3278, 3281, 3282, 3283, 3286, 3288, 3290, 3291, 3292, 3293, 3294, 3295, 3296, 3297, 3298, 3299, 3300, 3301, 3302, 3303, 3304, 3305, 3306, 3307, 3308, 3309, 3310, 3311, 3312, 3313, 3314, 3315, 3316, 3317, 3318, 3323, 3332, 3333, 3334, 3335, 3336, 3337, 3338, 3339, 3340, 3341, 3342, 3343, 3344, 3345, 3346, 3347, 3348, 3349, 3350, 3352, 3353, 3362, 3363, 3364, 3365, 3366, 3367, 3368, 3369, 3376, 3378, 3441, 3442, 3443, 3444, 3445, 3446, 3447, 3448, 3449, 3452, 3453, 3454, 3455, 3456, 3457, 3458, 3459, 3460, 3461, 3462, 3463, 3494, 3496, 3544, 3562, 3673, 3676, 3677, 3678, 3679, 3680, 3681, 3685, 3695, 3697, 3698, 3701, 3703, 3707, 3709, 3713,]
f_mask = [1420, 1438, 3593, 6336, 6500, 8653, 1256, 1429, 3145, 6336, 6500, 6509, 2, 1066, 5106, 6150, 6550, 2, 998, 3172, 6150, 8207, 8234, 2, 998, 3060, 5106, 6082, 8121, 8122, 1473, 3061, 5106, 6550, 8122, 8123, 55, 2786, 2938, 5159, 7997, 7999, 55, 2709, 2786, 6793, 7770, 7847, 130, 1507, 2987, 5232, 6584, 1718, 2926, 5232, 6584, 7529, 7987, 130, 2467, 3281, 3282, 5232, 7529, 8344, 130, 2987, 3282, 3283, 8047, 8345, 2114, 3512, 5440, 7179, 8210, 8572, 1474, 2422, 3148, 3512, 7484, 8210, 3064, 3512, 7484, 8126, 8200, 8572, 340, 3138, 5440, 8190, 8200, 8572, 256, 700, 5356, 5790, 5887, 8171, 700, 799, 1444, 5887, 6524, 8780, 3098, 3099, 3148, 7179, 7482, 8161, 8210, 1474, 2420, 3148, 3303, 6551, 7482, 814, 1282, 3303, 5902, 6551, 8365, 814, 1282, 1412, 2222, 6362, 6492, 814, 1412, 1472, 3051, 5902, 6549, 1474, 2422, 3051, 5902, 6551, 8113, 158, 3100, 3109, 5259, 5790, 8163, 8171, 700, 2421, 5259, 5790, 6524, 7483, 158, 2114, 3099, 3100, 7179, 8162, 158, 340, 2114, 5259, 5440, 7483, 1066, 1412, 1472, 1473, 6492, 6550, 1066, 2222, 3172, 6150, 6492, 7286, 844, 2420, 3096, 3098, 7482, 8160, 844, 1033, 2420, 3303, 5931, 8365, 642, 1049, 1497, 1769, 5733, 6574, 1049, 2488, 5733, 6133, 7550, 8052, 642, 2488, 3286, 3287, 5733, 8348, 642, 1769, 3287, 6844, 8349, 8350, 1432, 3145, 6509, 6512, 8207, 8234, 998, 1256, 2894, 3145, 6082, 7955, 8207, 1238, 3369, 3424, 5888, 6318, 8486, 800, 1238, 2815, 3625, 5888, 7876, 340, 2421, 3128, 7483, 8190, 8191, 1049, 1497, 2494, 3354, 3355, 6133, 8417, 2494, 2990, 6133, 7556, 7578, 8052, 1444, 3066, 3720, 5982, 8128, 8780, 1444, 2421, 3129, 6524, 8128, 8191, 2894, 3058, 3059, 6082, 8120, 8121, 2422, 3063, 7484, 8113, 8125, 8126, 1033, 1282, 1486, 6117, 6362, 8365, 1230, 2516, 2990, 6310, 7578, 7870, 2809, 2924, 6310, 7870, 7985, 7988, 1230, 2983, 6310, 7985, 8039, 8043, 1230, 2979, 2990, 7550, 8039, 8052, 1425, 3356, 3357, 3369, 6318, 6505, 8431, 1238, 1425, 2374, 2815, 6318, 7436, 1439, 2894, 3130, 7955, 8120, 8192, 1256, 1439, 3593, 6336, 6519, 7955, 2938, 2942, 3593, 6519, 7999, 8653, 1427, 1486, 2222, 6362, 6563, 7286, 1497, 3355, 3356, 6505, 6574, 8418, 1425, 1769, 6505, 6574, 6844, 7436, 1439, 2942, 3292, 6519, 8003, 8192, 1427, 1432, 3172, 6507, 7286, 8234, 1507, 2924, 2987, 7985, 8043, 8047, 1507, 2924, 2927, 6584, 7987, 7988, 1545, 2494, 3353, 3354, 7556, 8416, 1545, 2516, 6622, 7556, 7559, 7578, 1718, 2709, 2926, 2929, 6793, 7990, 1718, 2467, 2578, 6793, 7529, 7639, 7847, 2374, 3288, 6844, 7436, 8350, 8351, 3290, 3568, 3625, 7876, 8352, 8685, 2374, 2815, 3289, 7876, 8351, 8352, 2467, 3280, 3281, 7639, 8342, 8343, 2488, 2979, 3285, 3286, 7550, 8347, 2497, 2516, 2809, 7559, 7870, 8000, 2578, 3280, 3292, 7639, 8003, 8354, 2578, 2786, 2942, 7847, 7999, 8003, 2809, 2927, 2939, 7988, 7989, 8000, 2926, 2927, 2928, 7987, 7989, 7990, 2979, 2983, 3284, 3285, 8039, 8346, 2983, 3283, 3284, 8043, 8047, 3051, 3062, 6549, 8113, 8124, 8125, 1472, 1473, 6549, 8123, 8124, 3017, 3053, 3066, 3129, 8115, 8128, 3053, 3128, 3129, 3178, 8191, 8240, 3032, 3053, 8094, 8095, 8115, 8240, 2998, 3017, 3033, 8079, 8095, 8115, 1475, 3031, 3177, 6552, 8065, 8239, 1475, 3031, 3062, 3063, 8093, 8125, 1475, 2834, 3061, 3062, 6552, 8123, 8124, 2834, 2995, 3003, 6552, 7895, 8065, 2834, 3060, 3061, 6553, 7895, 8122, 1476, 1541, 2995, 6553, 7895, 8057, 1476, 1541, 3024, 6618, 7322, 8086, 1476, 3024, 3059, 3060, 6553, 8121, 3024, 3052, 3058, 3059, 8086, 8114, 2258, 3052, 3197, 7322, 8086, 2884, 3128, 3138, 3178, 7945, 8190, 2884, 3050, 3064, 3138, 8112, 8200, 2884, 2996, 7945, 8058, 8110, 8112, 3032, 3178, 7945, 8058, 8240, 3050, 3063, 3064, 8093, 8126, 3031, 3048, 3050, 8093, 8110, 8112, 8239, 2250, 3052, 3197, 7314, 8085, 8114, 8259, 3023, 3058, 3130, 8085, 8114, 8120, 1477, 3008, 6554, 8059, 8070, 8109, 1477, 3040, 3047, 8084, 8102, 8109, 1477, 3038, 3040, 6554, 8060, 8100, 2997, 2998, 3033, 6554, 8059, 8060, 1478, 1492, 2993, 2994, 6555, 8055, 8056, 1478, 3002, 3007, 8055, 8064, 8069, 1492, 3008, 6569, 8056, 8062, 8070, 2250, 3037, 3219, 3279, 7314, 8281, 2250, 3023, 3274, 8085, 8281, 8336, 1531, 3001, 3030, 3046, 6608, 8063, 8108, 1533, 3001, 3046, 3174, 6610, 8236, 1533, 3001, 3018, 3019, 8063, 8081, 1531, 2991, 3018, 8063, 8080, 1530, 2999, 3015, 3016, 6607, 8077, 8078, 1530, 1533, 3016, 3019, 6610, 1530, 3005, 6607, 6610, 8067, 8236, 2999, 6607, 8061, 8067, 8107, 1531, 2947, 2991, 3291, 6608, 8053, 8353, 2947, 3030, 6608, 8008, 8091, 8092, 2947, 3291, 8008, 8064, 8069, 3002, 3029, 8008, 8064, 8066, 8091, 2993, 3002, 3004, 8055, 8066, 8068, 1541, 3046, 3174, 6618, 8057, 8108, 2258, 3005, 3174, 6618, 7322, 8236, 2258, 3005, 3045, 3197, 8067, 8107, 8259, 3037, 3045, 7314, 8099, 8259, 2999, 3015, 3021, 8061, 8098, 3036, 3045, 8061, 8098, 8099, 8107, 3004, 3029, 3048, 3177, 8066, 8239, 2996, 3004, 3006, 3048, 8068, 8110, 2993, 2994, 3006, 3035, 8068, 8097, 2994, 2997, 3008, 3035, 8056, 8059, 3000, 3047, 8062, 8070, 8101, 8109, 2995, 3003, 3030, 8057, 8092, 8108, 3003, 3029, 3177, 8065, 8091, 8092, 2996, 3006, 3032, 8058, 8094, 8097, 2997, 3033, 3035, 8094, 8095, 8097, 2998, 3027, 8060, 8079, 8089, 8100, 3021, 3036, 3266, 3278, 8083, 8098, 8328, 8340, 3038, 3040, 3268, 8102, 8330, 8334, 3022, 3268, 3277, 8084, 8102, 8339, 3038, 3272, 3273, 8089, 8100, 8334, 897, 3027, 3273, 8089, 8335, 8628, 3023, 3130, 3292, 8192, 8336, 8354, 3022, 3049, 3275, 8111, 8337, 8339, 897, 3017, 3027, 3066, 5982, 8079, 3022, 3039, 3047, 8084, 8101, 8111, 3036, 3037, 3278, 3279, 8099, 8341, 3198, 3289, 3290, 8260, 8329, 8352, 3249, 3288, 3289, 8260, 8311, 8351, 3198, 3233, 3248, 3249, 8260, 8310, 3198, 3233, 3267, 8295, 8297, 8329, 3199, 3231, 3238, 8261, 8300, 8301, 3239, 3285, 8261, 8301, 8346, 8347, 3199, 3283, 3284, 8261, 8262, 8345, 8346, 3199, 3200, 3231, 8262, 8292, 8293, 3200, 3204, 3282, 8262, 8344, 8345, 3200, 3204, 3230, 8266, 8279, 8292, 3201, 3217, 3218, 8263, 8266, 8279, 3204, 3281, 8263, 8266, 8343, 8344, 3201, 3205, 8263, 8267, 8342, 8343, 3201, 3218, 3221, 8267, 8280, 3202, 3288, 8264, 8311, 8349, 8350, 3203, 3287, 8264, 8265, 8348, 8349, 3202, 3203, 3223, 3232, 8264, 8294, 3202, 3232, 3249, 8310, 8311, 3239, 3286, 8265, 8347, 8348, 3203, 3223, 3238, 3239, 8265, 8285, 8301, 3205, 3219, 3221, 3274, 8267, 8281, 8283, 3205, 3274, 3280, 8336, 8342, 8354, 3206, 3229, 3234, 3271, 8268, 8291, 3268, 3271, 3277, 8268, 8330, 8333, 3206, 3235, 3272, 8268, 8330, 8334, 3206, 3234, 3235, 8295, 8296, 8297, 3207, 3227, 3228, 8269, 8270, 8289, 8290, 3215, 3227, 3237, 8269, 8277, 8299, 3208, 3228, 3229, 3236, 8270, 8291, 3211, 3213, 3251, 8273, 8275, 8305, 8313, 3216, 3244, 8273, 8278, 8306, 8313, 3211, 3263, 8273, 8278, 8324, 8325, 3211, 3262, 8275, 8286, 8324, 3212, 3264, 3265, 8274, 8282, 8326, 8327, 3216, 8274, 8278, 8325, 8326, 3212, 3216, 3245, 8274, 8306, 8307, 3212, 3220, 3245, 3246, 8282, 3213, 3222, 3224, 8275, 8276, 8284, 8286, 3213, 3214, 3242, 3243, 8276, 8305, 3214, 3215, 3237, 8276, 8284, 3214, 3215, 3241, 3242, 8277, 8304, 3227, 3240, 3241, 8277, 8289, 8303, 3217, 3230, 3244, 3251, 8279, 8313, 3217, 3218, 3244, 8280, 8306, 8307, 3221, 3245, 3246, 8280, 8283, 8307, 8308, 3219, 3279, 8283, 8308, 8341, 3220, 8282, 8327, 8328, 8340, 3220, 3246, 3278, 8308, 8340, 8341, 3238, 3241, 8285, 8300, 8303, 8304, 3223, 3240, 8285, 8294, 8302, 8303, 3240, 3247, 8289, 8290, 8302, 8309, 3228, 3234, 8290, 8291, 8296, 8309, 3229, 3236, 3269, 3271, 8298, 8333, 3230, 3243, 3251, 8292, 8293, 8305, 3231, 3242, 3243, 8293, 8300, 8304, 3232, 3247, 3248, 8294, 8302, 8310, 3233, 3247, 3248, 8295, 8296, 8309, 3235, 3267, 3272, 3273, 8297, 8335, 3267, 3290, 3568, 8329, 8335, 8628, 3269, 3275, 3277, 8331, 8333, 8339, 3342, 3343, 3361, 8405, 8423, 8424, 3154, 3181, 3342, 3362, 8216, 8243, 8404, 8424, 3343, 3344, 8406, 8422, 8423, 3344, 3345, 3360, 8407, 8421, 8422, 3345, 3346, 3359, 8408, 8420, 8421, 3346, 3347, 3358, 8419, 8420, 3347, 3348, 3356, 3357, 8409, 8418, 8419, 3348, 3349, 3355, 8410, 8417, 8418, 3349, 3350, 3354, 8411, 8416, 8417, 3357, 3358, 3367, 8419, 8429, 8431, 3358, 3359, 3365, 8420, 8427, 8429, 3359, 3360, 3364, 8421, 8426, 8427, 3360, 3361, 8422, 8423, 8426, 3361, 3362, 3363, 3364, 8424, 8426, 3107, 3154, 3362, 3363, 8169, 8425, 3363, 3364, 3365, 3366, 8425, 8427, 8428, 3107, 3108, 8425, 8428, 3365, 3366, 3367, 3368, 8429, 8430, 3108, 3153, 3366, 8170, 8428, 8430, 3367, 3368, 3369, 8431, 8486, 256, 3153, 3368, 3424, 5356, 8215, 8430, 8486, 799, 800, 3625, 3720, 8685, 8780, 799, 800, 3424, 5356, 5887, 5888, 897, 3568, 3720, 5982, 8628, 8685, 1438, 2936, 2938, 6518, 7997, 8653, 3091, 3181, 3184, 8153, 8216, 8246, 3092, 3103, 8153, 8154, 8246, 3103, 3104, 3137, 8154, 8165, 8199, 3104, 3182, 3191, 8166, 8199, 8244, 3095, 3101, 3191, 8244, 8253, 3093, 3095, 3100, 3101, 8157, 8162, 8163, 3093, 3099, 3183, 8155, 8161, 8162, 3094, 3098, 3183, 8160, 8161, 8245, 3101, 3109, 3111, 8163, 8173, 8253, 3111, 3152, 3191, 8166, 8214, 8253, 3104, 3150, 3152, 8165, 8166, 8212, 3103, 3184, 8165, 8212, 8246, 3150, 3154, 3184, 8169, 8212, 8216, 3107, 3108, 3150, 3152, 8169, 8170, 8214, 3111, 3153, 8170, 8173, 8214, 8215, 256, 3109, 8171, 8173, 8215]
mask = {'v_mask':v_mask, 'f_mask':f_mask}
v, f = params_to_model(shape=model_kwargs['shape'].cuda().to(th.float32), 
                exp=model_kwargs['exp'].cuda().to(th.float32), 
                pose=model_kwargs['pose'].cuda().to(th.float32), 
                cam=model_kwargs['cam'].cuda().to(th.float32),
                lights=model_kwargs['light'].reshape(-1, 9, 3).cuda().to(th.float32),
                mask=mask
)


print(v.shape, f.shape)
v_mask = v[:, mask, :][[0]]
f_mask = f[:, rm_faces, :]
print(v_mask.shape, f_mask.shape)

def face_vertices(vertices, faces):
    """ 
    :param vertices: [batch size, number of vertices, 3]
    :param faces: [batch size, number of faces, 3]
    :return: [batch size, number of faces, 3, 3]
    """
    assert (vertices.ndimension() == 3)
    assert (faces.ndimension() == 3)
    assert (vertices.shape[0] == faces.shape[0])
    assert (vertices.shape[2] == 3)
    assert (faces.shape[2] == 3)

    bs, nv = vertices.shape[:2]
    bs, nf = faces.shape[:2]
    device = vertices.device
    faces = faces + (th.arange(bs, dtype=th.int32).to(device) * nv)[:, None, None]
    print(faces)
    print(faces.shape)
    print(vertices.shape)
    vertices = vertices.reshape((bs * nv, 3))
    print(vertices.shape)
    # assert False
    # pytorch only supports long and byte tensors for indexing
    return vertices[faces.long()]

def vertex_normals(vertices, faces):
    """
    :param vertices: [batch size, number of vertices, 3]
    :param faces: [batch size, number of faces, 3]
    :return: [batch size, number of vertices, 3]
    """
    assert (vertices.ndimension() == 3)
    assert (faces.ndimension() == 3)
    assert (vertices.shape[0] == faces.shape[0])
    assert (vertices.shape[2] == 3)
    assert (faces.shape[2] == 3)
    bs, nv = vertices.shape[:2]
    bs, nf = faces.shape[:2]
    device = vertices.device
    normals = torch.zeros(bs * nv, 3).to(device)
    print(faces)
    assert False

    faces = faces + (torch.arange(bs, dtype=torch.int32).to(device) * nv)[:, None, None] # expanded faces
    vertices_faces = vertices.reshape((bs * nv, 3))[faces.long()]

    faces = faces.reshape(-1, 3)
    vertices_faces = vertices_faces.reshape(-1, 3, 3)

    normals.index_add_(0, faces[:, 1].long(), 
                       torch.cross(vertices_faces[:, 2] - vertices_faces[:, 1], vertices_faces[:, 0] - vertices_faces[:, 1]))
    normals.index_add_(0, faces[:, 2].long(), 
                       torch.cross(vertices_faces[:, 0] - vertices_faces[:, 2], vertices_faces[:, 1] - vertices_faces[:, 2]))
    normals.index_add_(0, faces[:, 0].long(),
                       torch.cross(vertices_faces[:, 1] - vertices_faces[:, 0], vertices_faces[:, 2] - vertices_faces[:, 0]))

    normals = F.normalize(normals, eps=1e-6, dim=1)
    normals = normals.reshape((bs, nv, 3))
    # pytorch only supports long and byte tensors for indexing
    return normals

face_vertices(v[[0]], f_mask)
vertex_normals(v[[0]], self.faces.expand(batch_size, -1, -1)); 


creating the FLAME Decoder


/home/mint/miniconda3/envs/3dr_sampling_deca/lib/python3.8/site-packages/pytorch3d/io/obj_io.py:457: UserWarning: Mtl file does not exist: /home/mint/guided-diffusion/model_3d/FLAME/data/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


torch.Size([2, 253, 3])
torch.Size([2, 253, 3])
torch.Size([1, 1520, 3])
torch.Size([1, 1520, 3, 3])
DONE face_vertices


/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [49,0,0], thread: [32,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [49,0,0], thread: [33,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [49,0,0], thread: [34,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [49,0,0], thread: [35,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [49,0,0], thread: [36,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [49,0,0

RuntimeError: CUDA error: device-side assert triggered

In [4]:
from importlib import reload
def params_to_model(shape, exp, pose, cam, lights, mask):
    
    from model_3d.FLAME.config import cfg as flame_cfg
    import model_3d.FLAME.FLAME as FLAME
    import model_3d.FLAME.utils.renderer as renderer_lib
    import model_3d.FLAME.utils.util as util
    reload(renderer_lib)
    reload(FLAME)

    flame = FLAME.FLAME(flame_cfg.model).cuda()
    verts, landmarks2d, landmarks3d = flame(shape_params=shape, 
            expression_params=exp, 
            pose_params=pose)
    renderer = renderer_lib.SRenderY(image_size=256, obj_filename=flame_cfg.model.topology_path, uv_size=flame_cfg.model.uv_size).cuda()

    verts = verts[:, mask['v_mask'], :]
    ## projection
    landmarks2d = util.batch_orth_proj(landmarks2d, cam)[:,:,:2]; landmarks2d[:,:,1:] = -landmarks2d[:,:,1:]#; landmarks2d = landmarks2d*self.image_size/2 + self.image_size/2
    landmarks3d = util.batch_orth_proj(landmarks3d, cam); landmarks3d[:,:,1:] = -landmarks3d[:,:,1:] #; landmarks3d = landmarks3d*self.image_size/2 + self.image_size/2
    trans_verts = util.batch_orth_proj(verts, cam); trans_verts[:,:,1:] = -trans_verts[:,:,1:]

    ## rendering
    shape_images = renderer.render_shape(verts, trans_verts, lights=lights, mask=mask)

    # opdict = {'verts' : verts,}
    # os.makedirs('./rendered_obj', exist_ok=True)
    # save_obj(renderer=renderer, filename=(f'./rendered_obj/{i}.obj'), opdict=opdict)
    shape_images = shape_images.mul(255).add_(0.5).clamp_(0, 255)
    return {"shape_images":shape_images, "landmarks2d":landmarks2d, "landmarks3d":landmarks3d}

f_mask = np.load('./faces_index.pkl', allow_pickle=True, encoding='latin1')
v_mask = np.load('./FLAME_masks.pkl', allow_pickle=True, encoding='latin1')

out = params_to_model(shape=model_kwargs['shape'].cuda().to(th.float32), 
                exp=model_kwargs['exp'].cuda().to(th.float32), 
                pose=model_kwargs['pose'].cuda().to(th.float32), 
                cam=model_kwargs['cam'].cuda().to(th.float32),
                lights=model_kwargs['light'].reshape(-1, 9, 3).cuda().to(th.float32),
                mask={
                    'v_mask':v_mask['eye_region'].tolist(),
                    'f_mask':f_mask['eye_region'].tolist()
                }
                
)
show(torchvision.utils.make_grid(out['shape_images'].cpu())/255)

creating the FLAME Decoder


/home/mint/miniconda3/envs/3dr_sampling_deca/lib/python3.8/site-packages/pytorch3d/io/obj_io.py:457: UserWarning: Mtl file does not exist: /home/mint/guided-diffusion/model_3d/FLAME/data/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


torch.Size([1, 4512, 3])
torch.Size([1, 4512, 3, 3])


/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [141,0,0], thread: [96,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [141,0,0], thread: [97,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [141,0,0], thread: [98,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [141,0,0], thread: [99,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [141,0,0], thread: [100,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:84: operator(): block: [

RuntimeError: CUDA error: device-side assert triggered